In [1]:
#- Basic imports
import numpy as np
import bokeh.plotting as bk
bk.output_notebook()

Loading BokehJS ...

In [2]:
from astropy.time import Time
from astropy.time import TimezoneInfo
import astropy.units as u
from datetime import tzinfo
from datetime import datetime
from astropy import coordinates
from astropy.coordinates import EarthLocation

In [ ]:
from astropy.table import Table
tiles = Table.read('desi-tiles.fits')

#- Read exposures
exposures = Table.read('exposures.fits')

#- Separate calibration exposures
all_exposures = exposures[exposures['PROGRAM'] != 'CALIB']
all_calibs = exposures[exposures['PROGRAM'] == 'CALIB']

#- Filter exposures to just this night and adds columns DATETIME and MJD_hour
night_zero_exps = find_night(all_exposures, night_zero)
night_zero_calibs = find_night(all_calibs, night_zero)

In [26]:
def generateNights(initial):
    """
    Generates a list of times for each night at Greenwich midnight through a 10 year period
    
    Args:
        initial : an initial Time object at midnight
    
    Returns a numpy array of Time objects
    """
    # convert Greenwich midnight to local midnight
    initial = initial + (-7) * u.hour
    stop = initial + 10 * u.year
    
    times = np.array([])
    curr = initial
    while (stop.datetime() != stop):
        times = np.append(curr)
        curr = curr + 1 * u.day
    
    return times

In [27]:
import ephem
def config_ephem(night):
    """
    Returns a ephem.Observer object which is configured to a specific night 
    at the conditions atop Kitt Peak
    
    Args:
        night : a Time object of a night of observation at local midnight
    """
    mayall = ephem.Observer()
    mayall.lat = str(31.963972222)
    mayall.lon = str(-111.599336111)
    mayall.elevation = 2120
    mayall.temp = 5
    mayall.pressure = 78318
    mayall.date = night
    return mayall

In [9]:
def get_morning_times(mayall):
    """
    Returns times for the sunrise and twelve and eighteen degree 
    twilights for the morning after a particular night of observation
    
    Args:
        mayall : ephem.Observer object configured for Kitt Peak on a given night at midnight
    
    Returns 3 Time objects
    """
    
    #get sunrise
    t_sunrise_prev = mayall.previous_rising(ephem.Sun(), use_center=False)
    sunrise = Time(t_sunrise_prev.datetime(), format='datetime')
    sunrise_next = sunrise + 1 * u.hour
    local_sunrise = sunrise_next + (-7) * u.hour
    
    # get 18-deg twilight
    mayall.horizon = '-18'
    t_twilight_one = mayall.previous_rising(ephem.Sun(), use_center=True)
    t_one = Time(t_twilight_one.datetime(), format='datetime')
    t_one_next = t_one + 1 * u.hour
    local_twilight_one = t_one_next + (-7) * u.hour
    
    # get 12-deg twilight
    mayall.horizon = '-12'
    t_twilight_two = mayall.previous_rising(ephem.Sun(), use_center=True)
    t_two = Time(t_twilight_two.datetime(), format='datetime')
    t_two_next = t_two + 1 * u.hour
    local_twilight_two = t_two_next + (-7) * u.hour
    
    return local_sunrise, local_twilight_one, local_twilight_two

In [10]:
def get_evening_times(mayall):
    """
    Returns times for the sunset and twelve and eighteen degree 
    twilights for the evening before a particular night of observation
    
    Args:
        mayall : ephem.Observer object configured for Kitt Peak on a given night at midnight
    
    Returns 3 Time objects    
    """
    
    #get sunset
    t_sunset = mayall.next_setting(ephem.Sun(), use_center=False)
    sunset = Time(t_sunset.datetime(), format='datetime')
    local_sunset = sunset + (-7) * u.hour
    
    # get 18-deg twilight
    mayall.horizon = '-18'
    t_twilight_one = mayall.next_setting(ephem.Sun(), use_center=True)
    t_one = Time(t_twilight_one.datetime(), format='datetime') 
    local_twilight_one = t_one + (-7) * u.hour
    
    # get 12-deg twilight
    mayall.horizon = '-12'
    t_twilight_two = mayall.next_setting(ephem.Sun(), use_center=True)
    t_two = Time(t_twilight_two.datetime(), format='datetime')
    local_twilight_two = t_two + (-7) * u.hour

    return local_sunset, local_twilight_one, local_twilight_two

In [8]:
night_zero = '20191203'

In [9]:
mayall = config_ephem(night_zero)

In [52]:
def get_vertical_times(night):
    """
    Calculates the sunset, evening 18-deg twilight, evening 12-deg twilight, 
    sunrise, morning 18-deg twilight, and morning 12-deg twilight for a given night of observation
    
    Args:
        night : Time object at local midnight on a specific night
    
    Returns an array of datetime objects
    """
    mayall = config_ephem(night)
    sunset, eve_t1, eve_t2 = get_evening_times(mayall)
    sunrise, morn_t1, morn_t2 = get_morning_times(mayall)
    return [sunset, eve_t1, eve_t2, sunrise, morn_t1, morn_t2]

In [ ]:
night_zero = '20191203'
times = get_vertical_times(night_zero)
print(times)

In [44]:
def convertToStringNight(dt):
    """
    Converts a datetime into a string representation of the night = YYYYMMDD
    
    Args:
        dt : a datetime object
    Returns a string
    """
    year = str(dt.year)
    month = dt.month
    if month < 10:
        month = '0' + str(month)
    else:
        month = str(month)
    day = dt.day
    if day < 10:
        day = '0' + str(day)
    else:
        day = str(day)
    return year + month + day

In [45]:
import csv

def writeTimes(output_directory, filename):
    """
    Writes the sunset, sunrise, and four twilight times to a file
    
    Args:
        output_directory : directory to write the file to
        filename : tsv filename
    """
    times = generateNights(Time(datetime(2019, 1, 1), format='datetime'))
    
    f = open(output_directory + "\\" + filename, 'wt')
    with f as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(['night', 'sunset', 'eve18deg', 'eve12deg', 'sunrise', 'mor18deg', 'mor12deg'])
        for n in times:
            night = convertToStringNight(n.datetime())
            row = [night]
            times = get_vertical_times(n)
            row = row + times
            tsv_writer.writerow(row)
    file.close()

In [32]:
def get_localtime(night):
    """
    Returns the local time at midnight on the given NIGHT
    
    Args:
        night : night = YEARMMDD of sunset
    
    Returns a Time object
    """
    #- Re-formats night into YYYY-MM-DD HH:MM:SS 
    iso_format = night[:4] + '-' + night[4:6] + '-' + night[6:] + ' 00:00:00'
    t_midnight = Time(iso_format, format='iso') + 24*u.hour
    #- Set timezone
    t_local = t_midnight + (-7)*u.hour
    return t_local

In [53]:
def readTimes(input_directory, filename, night):
    """
    Reads the sunset, sunrise, and four twilight times from a file for a given night
    
    Args:
        input_directory : string of the input directory
        filename : string of the filename
        night : string representation (YEARMMDD) of a night of observation
    
    Returns an array of Time objects
    """
    times = []
    f = input_directory + "\\" + filename
    with open(f) as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        times = row[night]
    f.close()
    return [t.datetime() for t in times]

In [ ]:
# plot on the timeseries plots using Span